# Variational Quantum Classifier – Per Station
*Generated 2025-08-07 15:57*

This notebook trains one VQC per hydrological station using:
* Balanced flood / non‑flood undersampling (50/50)
* PCA to at most 15 qubits
* 2 Strongly‑Entangling layers, 30 epochs, mini‑batch 128
* `OMP_NUM_THREADS=1` to avoid OpenMP crashes on Apple Silicon


In [1]:
import os, warnings
os.environ["OMP_NUM_THREADS"]="4"   # keep MKL/OpenMP under control
warnings.filterwarnings("ignore")


In [2]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix


In [3]:
# --- USER CONFIG -----------------------------------------------------
PROJECT_ROOT = Path.cwd() if (Path.cwd()/'data').exists() else Path.cwd().parent
DATA_RAW = PROJECT_ROOT/'data'/'raw'  # adjust if needed
STATIONS = ['D08A071','D08A084','D08A115']
SEED = 42
PERCENTILE = 0.92
ROLL_WINDOWS=[3,6,12]
LAG_HRS=range(1,13)
API_WINDOW=24*7
np.random.seed(SEED)


In [4]:
def find_csv(code):
    p = DATA_RAW / f'station_{code}'
    files = list(p.glob('*.csv'))
    assert len(files)==1, f'{code}: CSV not found'
    return files[0]
def load_station(code):
    df = pd.read_csv(find_csv(code))
    df['datetime'] = pd.to_datetime(df['saatlik'])
    df = (df.rename(columns={'yagis_toplam':'rain_mm','qdeger':'discharge_cms'})
            .set_index('datetime').sort_index()
            .resample('h').agg({'rain_mm':'sum','discharge_cms':'mean'}))
    return df

def build_features(df):
    st=df.copy()
    for w in ROLL_WINDOWS:
        st[f'rain_sum_{w}h']=st['rain_mm'].rolling(w,1).sum()
    for l in LAG_HRS:
        st[f'dis_lag_{l}h']=st['discharge_cms'].shift(l)
    st['dis_rate_1h']=st['discharge_cms'].diff(1)
    st['dis_rate_3h']=st['discharge_cms'].diff(3)
    st['API7']=st['rain_mm'].rolling(API_WINDOW,1).sum()
    st.dropna(inplace=True)
    thr=st['discharge_cms'].quantile(PERCENTILE)
    st['flood']=(st['discharge_cms']>thr).astype(int)
    return st


In [5]:

def train_vqc(X_tr,y_tr,X_val,y_val,n_qubits,epochs=30,batch=64,patience=5):
    dev= qml.device('lightning.qubit', wires=n_qubits)
    @qml.qnode(dev, interface='autograd', diff_method='adjoint')
    def circuit(x,w):
        qml.templates.AngleEmbedding(x, wires=range(n_qubits))
        qml.templates.StronglyEntanglingLayers(w, wires=range(n_qubits))
        return qml.expval(qml.PauliZ(0))
    w_shape=qml.templates.StronglyEntanglingLayers.shape(2,n_wires=n_qubits)
    w=pnp.random.random(w_shape, requires_grad=True)
    opt=qml.AdamOptimizer(0.01)
    eps=1e-7
    def bce(weights, xb, yb):
        logits=pnp.array([circuit(xi,weights) for xi in xb])
        p   =(logits+1)/2          # map -1..1 → 0..1
        return -pnp.mean(yb*pnp.log(p+eps)+(1-yb)*pnp.log(1-p+eps))
    best=1e9; wait=0
    for ep in range(epochs):
        perm=np.random.permutation(len(X_tr))
        for s in range(0,len(X_tr),batch):
            idx=perm[s:s+batch]
            w=opt.step(lambda ww: bce(ww,X_tr[idx],y_tr[idx]), w)
        val=bce(w,X_val,y_val)
        if val<best-1e-4: best=val; wait=0
        else:
            wait+=1
            if wait>=patience: break
        if ep%5==0: print(f"epoch {ep:2d} val {val:.4f}")
    return w,circuit


In [6]:

summary=[]
for code in STATIONS:
    print(f"\n=== {code} ===")
    df=build_features(load_station(code))
    feats=[c for c in df.columns if c.startswith(('rain_sum','dis_lag','dis_rate','API'))]
    X=df[feats].values
    X=(X-X.mean(0))/X.std(0)
    y=df['flood'].values
    pos=np.where(y==1)[0]
    neg=np.random.choice(np.where(y==0)[0],size=len(pos),replace=False)
    idx=np.concatenate([pos,neg])
    X_bal,y_bal=X[idx],y[idx]
    n_qubits=min(10,X_bal.shape[1])
    X_bal=PCA(n_components=n_qubits,random_state=SEED).fit_transform(X_bal)
    X_tr,X_tmp,y_tr,y_tmp=train_test_split(X_bal,y_bal,test_size=0.4,random_state=SEED,stratify=y_bal)
    X_val,X_te,y_val,y_te=train_test_split(X_tmp,y_tmp,test_size=0.5,random_state=SEED,stratify=y_tmp)
    X_tr=pnp.array(X_tr,dtype=np.float32); y_tr=pnp.array(y_tr,dtype=np.float32)
    X_val=pnp.array(X_val,dtype=np.float32); y_val=pnp.array(y_val,dtype=np.float32)
    w,circ=train_vqc(X_tr,y_tr,X_val,y_val,n_qubits)
    probs=np.array([circ(x,w) for x in X_te])
    y_pred=(probs>0).astype(int)   # threshold 0
    acc=accuracy_score(y_te,y_pred)
    prec=precision_score(y_te,y_pred,zero_division=0)
    rec=recall_score(y_te,y_pred)
    cm=confusion_matrix(y_te,y_pred,labels=[0,1])
    print(cm)
    summary.append([code,acc,prec,rec])
print("\nSummary")
for s in summary:
    print(f"{s[0]}  Acc {s[1]:.3f}  Prec {s[2]:.3f}  Rec {s[3]:.3f}")



=== D08A071 ===
epoch  0 val 0.6848
epoch  5 val 0.2604
epoch 10 val 0.2596
[[357  15]
 [ 60 312]]

=== D08A084 ===
epoch  0 val 0.6479
epoch  5 val 0.4695
epoch 10 val 0.4686
epoch 15 val 0.4682
[[472  26]
 [270 228]]

=== D08A115 ===
epoch  0 val 0.6944
epoch  5 val 0.4900
epoch 10 val 0.4796
epoch 15 val 0.4713
epoch 20 val 0.4701
[[321  25]
 [160 186]]

Summary
D08A071  Acc 0.899  Prec 0.954  Rec 0.839
D08A084  Acc 0.703  Prec 0.898  Rec 0.458
D08A115  Acc 0.733  Prec 0.882  Rec 0.538
